#  Классификация документов с помощью библиотеки fastText

### Установка fastText

В папку с настоящим "ноутбуком".
1. git clone https://github.com/facebookresearch/fastText.git
2. cd fastText
3. make

In [1]:
from sklearn.datasets import fetch_20newsgroups
import re
import nltk

## Загрузка датасета 20 newsgroups

In [2]:
categories = [
        'talk.politics.misc',
        'talk.religion.misc',
        'comp.graphics',
        'sci.space',
        'rec.autos',
    ]
remove = ('headers', 'footers', 'quotes')

print("Loading 20 newsgroups dataset for categories:")
print(categories if categories else "all")

data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42, remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42, remove=remove)
print('data loaded')

Loading 20 newsgroups dataset for categories:
['talk.politics.misc', 'talk.religion.misc', 'comp.graphics', 'sci.space', 'rec.autos']
data loaded


## Предобработка текста

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/annakupriyanova/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/annakupriyanova/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/annakupriyanova/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [39]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

def preprocess_text(text):
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    # stem the words
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    # lemmatize
    # lemmatizer = WordNetLemmatizer()
    # words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [35]:
preprocess_text(data_train.data[0])

'wife looked drove one last fall model wayyyyyyyyy underpowered could imagine driving mountain colorado anything approaching highway speed read new model newer improved hp engine quite serious laughed salesman face said broken feel powerful used driving jeep engine believe land cruiser land yacht same also underpowered personal opinion big car roomy nothing spectacular'

## Структурирование датасета в формате библиотеки fasttext

Формат библиотеки fasttext: каждая строка содержит запись вида \__label\__{label_id} text. Например, \__label\__1 computers will never die

In [13]:
def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"\d", "", string)
    string = re.sub(r"\n\t", " ", string)
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

In [22]:
def fasttext_format_dataset(data, target, filename):
    with open(filename, 'w') as fout:
        for x, y in zip(data, target):
            # x = strip_formatting(x)
            x = preprocess_text(x)
            line = '__label__{} {}'.format(y, x)
            fout.write(line + '\n')

In [40]:
# create train dataset in fasttext format
fasttext_format_dataset(data_train.data, data_train.target, 'ft_train.txt')

# create test dataset in fasttext format
fasttext_format_dataset(data_test.data, data_test.target, 'ft_test.txt')

print('Train and test datasets are ready.')

Train and test datasets are ready.


## Обучение и тестирование модели fasttext

In [41]:
%%bash
./fastText/fasttext supervised -input ft_train.txt -output ft_model

Read 0M words
Number of words:  21309
Number of labels: 5
Progress: 100.0% words/sec/thread:  494845 lr:  0.000000 loss:  1.595745 ETA:   0h 0m


In [42]:
%%bash
./fastText/fasttext test ft_model.bin ft_test.txt

N	1740
P@1	0.329
R@1	0.329


### Эксперименты с параметрами обучения

In [92]:
%%bash
./fastText/fasttext supervised -input ft_train.txt -output ft_model -epoch 100 -lr 0.8 -dim 100

Read 0M words
Number of words:  21309
Number of labels: 5
Progress: 100.0% words/sec/thread:  766357 lr:  0.000000 loss:  0.351176 ETA:   0h 0m


In [93]:
%%bash
./fastText/fasttext test ft_model.bin ft_test.txt

N	1740
P@1	0.781
R@1	0.781


### Переобучим модель, добавив биграммы

In [94]:
%%bash
./fastText/fasttext supervised -input ft_train.txt -output ft_model_2grams -epoch 100 -lr 0.8 -dim 100 -wordNgrams 2

Read 0M words
Number of words:  21309
Number of labels: 5
Progress: 100.0% words/sec/thread:  397281 lr:  0.000000 loss:  0.458082 ETA:   0h 0m


In [95]:
%%bash
./fastText/fasttext test ft_model_2grams.bin ft_test.txt

N	1740
P@1	0.779
R@1	0.779


## Классификация новых текстов

In [96]:
print('Categories:')
for i in range(5):
    print(f'{i} - {data_train.target_names[i]}')

Categories:
0 - comp.graphics
1 - rec.autos
2 - sci.space
3 - talk.politics.misc
4 - talk.religion.misc


In [99]:
preprocess_text("Computers are cool stuff. I'd  like to work with software.")

'comput cool stuff like work softwar'

In [100]:
%%bash
./fastText/fasttext predict ft_model.bin -
comput cool stuff like work softwar

__label__0


### Топ-3 предсказанных категорий

In [101]:
preprocess_text("Russian President signs new migration policy concept.")

'russian presid sign new migrat polici concept'

In [102]:
%%bash
./fastText/fasttext predict ft_model.bin - 3
russian presid sign new migrat polici concept

__label__3 __label__1 __label__2


# Задание на практику

Поменять параметры классификатора и/или алгоритм предобработки датасета, чтобы добиться более высоких показателей точности и полноты.